### RHPI analysis project

## HPI data

We start by loading the HPI data and the LAD to Region lookup table and joining the appropriate regions onto our HPI table

In [4]:
### import pandas as pd
import pandas as pd
import sqlite3

#loading dataset
df = pd.read_csv("UK-HPI-full-file-2025-03.csv")

#checking all data types
print(df.dtypes)

#connecting to SQLite database (creates new if nonexistent)
conn = sqlite3.connect("house_prices_1.db")
cursor = conn.cursor()

#writing DataFrame to SQLite (replacing if table exists)
try:
    df.to_sql(name='uk_hpi', con=conn, if_exists='replace', index=False)
    print("Data successfully loaded to SQLite!")
except Exception as e:
    print(f"Error: {e}")

#updating 2 incorrect district codes (Barnsley,Sheffield) (see line 103)

update = """ UPDATE uk_hpi
SET AreaCode = CASE AreaCode
    WHEN 'E08000038' THEN 'E08000016'
    WHEN 'E08000039' THEN 'E08000019'
    ELSE AreaCode 
END
WHERE AreaCode IN ('E08000038', 'E08000039');
"""
cursor.execute(update)
conn.commit()

#loading LAD to Region lookup table
#We will this to assign general regions (North West etc..) to regions in the HPI dataset

df1 = pd.read_csv("Local_Authority_District_to_Region_(December_2023)_Lookup_in_England.csv")

try:
    df1.to_sql(name='LAD_Regs', con=conn, if_exists='replace', index=False)
    print("Data successfully loaded to SQLite!")
except Exception as e:
    print(f"Error: {e}")


query1 = """
SELECT COUNT(DISTINCT RegionName)
FROM uk_hpi """

query2 = """
SELECT COUNT(DISTINCT LAD23NM)
FROM LAD_Regs """

#comparing number of regions in datasets
print(pd.read_sql_query(query1, conn))
print(pd.read_sql_query(query2, conn))

#disconnect between counts as LAD_reg only contains English regions

#for this analysis we will only compare English regions so want to clear extras from HPI

Joined_table = """ DROP TABLE IF EXISTS joined_tab;
CREATE TABLE joined_tab AS
SELECT uk_hpi.*, LAD_Regs.RGN23NM AS Standard_Region_Name
FROM uk_hpi
INNER JOIN LAD_Regs 
ON uk_hpi.AreaCode = LAD_Regs.LAD23CD;
"""

cursor.executescript(Joined_table)

check_join_0 = """
SELECT Date, RegionName, "Index"
FROM joined_tab """

df4 = pd.read_sql_query(check_join_0, conn)
df4.to_csv('HPI99.csv', index=False)

#checking no of districts in new table
query3 = """
SELECT COUNT(DISTINCT RegionName)
FROM joined_tab
"""

#3 missing districts in joined_tabs, running SQL code to identify them
query4 = """
SELECT
    lku.LAD23NM,
    lku.LAD23CD
FROM
    LAD_Regs AS lku
LEFT JOIN
    uk_hpi AS hpi
ON
    lku.LAD23CD = hpi.AreaCode
WHERE
    hpi.AreaCode IS NULL """

print(pd.read_sql_query(query3, conn))
print(pd.read_sql_query(query4, conn))

# identified 2 Districts with differing codes  between tables and 1 with no data(Isles of Scilly) so will make adjustment to this at start of code
# in all cases LAD_Regs has correct code so will adjust UK_HPI (see line 18)

Date                       object
RegionName                 object
AreaCode                   object
AveragePrice                int64
Index                     float64
IndexSA                   float64
1m%Change                 float64
12m%Change                float64
AveragePriceSA            float64
SalesVolume               float64
DetachedPrice             float64
DetachedIndex             float64
Detached1m%Change         float64
Detached12m%Change        float64
SemiDetachedPrice         float64
SemiDetachedIndex         float64
SemiDetached1m%Change     float64
SemiDetached12m%Change    float64
TerracedPrice             float64
TerracedIndex             float64
Terraced1m%Change         float64
Terraced12m%Change        float64
FlatPrice                 float64
FlatIndex                 float64
Flat1m%Change             float64
Flat12m%Change            float64
CashPrice                 float64
CashIndex                 float64
Cash1m%Change             float64
Cash12m%Change

### HPI rebasing

We have now added regions to the HPI data.

Next step is to create 3 rebased HPI tables (bases = 2005,2015,2020). We need these to provide the option of ~ last 5/10/20 years in our interactive time series graph.

In [5]:
view_2015 = """
DROP VIEW IF EXISTS uk_hpi_2015;
CREATE VIEW uk_hpi_2015 AS
SELECT
    t1.Date,    
    t1.RegionName AS District,
    t1.Standard_Region_Name AS Region,
    (t1."Index" / base."Index") * 100 AS "Index",
    (t1.DetachedIndex / base.DetachedIndex) * 100 AS DetachedIndex,
    (t1.SemiDetachedIndex / base.SemiDetachedIndex) * 100 AS SemiDetachedIndex,
    (t1.TerracedIndex / base.TerracedIndex) * 100 AS TerracedIndex,
    (t1.FlatIndex / base.FlatIndex) * 100 AS FlatIndex
FROM joined_tab AS t1 
JOIN uk_hpi AS base
  ON t1.RegionName = base.RegionName
 AND base.Date = '2015-01-01';
"""

cursor.executescript(view_2015)
conn.commit()

# Checking updated table
check_2015 = """ SELECT *
FROM uk_hpi_2015
WHERE Date ='2015-08-01'
LIMIT 5 """

#print(pd.read_sql_query(check_2015, conn))

# I will now repeat this method to create a view with January 2020 base and a view with January 2006 base

view_2020 = """
DROP VIEW IF EXISTS uk_hpi_2020;
CREATE VIEW uk_hpi_2020 AS
SELECT
    t1.Date,    
    t1.RegionName AS District,
    t1.Standard_Region_Name AS Region,
    (t1."Index" / base."Index") * 100 AS "Index",
    (t1.DetachedIndex / base.DetachedIndex) * 100 AS DetachedIndex,
    (t1.SemiDetachedIndex / base.SemiDetachedIndex) * 100 AS SemiDetachedIndex,
    (t1.TerracedIndex / base.TerracedIndex) * 100 AS TerracedIndex,
    (t1.FlatIndex / base.FlatIndex) * 100 AS FlatIndex
FROM joined_tab AS t1 
JOIN uk_hpi AS base
  ON t1.RegionName = base.RegionName
 AND base.Date = '2020-01-01';
"""

cursor.executescript(view_2020)
conn.commit()


view_2005 = """
DROP VIEW IF EXISTS uk_hpi_2005;
CREATE VIEW uk_hpi_2005 AS
SELECT
    t1.Date,    
    t1.RegionName AS District,
    t1.Standard_Region_Name AS Region,
    (t1."Index" / base."Index") * 100 AS "Index",
    (t1.DetachedIndex / base.DetachedIndex) * 100 AS DetachedIndex,
    (t1.SemiDetachedIndex / base.SemiDetachedIndex) * 100 AS SemiDetachedIndex,
    (t1.TerracedIndex / base.TerracedIndex) * 100 AS TerracedIndex,
    (t1.FlatIndex / base.FlatIndex) * 100 AS FlatIndex
FROM joined_tab AS t1 
JOIN uk_hpi AS base
  ON t1.RegionName = base.RegionName
 AND base.Date = '2005-01-01';
"""

cursor.executescript(view_2005)
conn.commit()


### CPIH rebasing

Loading CPIH data and rebasing in very similar way to HPI

In [6]:
#loading CPIH table
#Used excel to change the form of date data to match HPI table, eg, Jan-04 to 01/01/04 and to rearrange table from pivot to match HPI data

df2 = pd.read_excel("CPIH1.xlsx",  engine="openpyxl")

try:
    df2.to_sql(name='CPIH_2015', con=conn, if_exists='replace', index=False)
    print("Data successfully loaded to SQLite!")
except Exception as e:
    print(f"Error: {e}")


print(df2)


#In order to calculate RHPI for our adjusted bases we must first produce CPIH tables with adjusted bases

CPIH_2015_J = """
DROP VIEW IF EXISTS CPIH_2015_J;
CREATE VIEW CPIH_2015_J AS
SELECT STRFTIME('%Y-%m-%d',A) AS A, (B / 99.2) * 100 AS B
FROM CPIH_2015"""

cursor.executescript(CPIH_2015_J)
conn.commit()

check_1 = """ SELECT * FROM CPIH_2015_J """
#print(pd.read_sql_query(check_1, conn))


CPIH_2020 = """
DROP VIEW IF EXISTS CPIH_2020;
CREATE VIEW CPIH_2020 AS
SELECT STRFTIME('%Y-%m-%d',A) AS A, (B / 108.3) * 100 AS B
FROM CPIH_2015""" 

cursor.executescript(CPIH_2020)
conn.commit()

check_2 = """ SELECT * FROM CPIH_2020 """
#print(pd.read_sql_query(check_2, conn))

CPIH_2005 = """
DROP VIEW IF EXISTS CPIH_2005;
CREATE VIEW CPIH_2005 AS
SELECT STRFTIME('%Y-%m-%d',A) AS A, (B / 78.3) * 100 AS B
FROM CPIH_2015""" 

cursor.executescript(CPIH_2005)
conn.commit()

check_3 = """ SELECT * FROM CPIH_2005 """
#print(pd.read_sql_query(check_3, conn))

Data successfully loaded to SQLite!
             A      B
0   2004-01-01   77.0
1   2004-02-01   77.2
2   2004-03-01   77.3
3   2004-04-01   77.6
4   2004-05-01   77.9
..         ...    ...
252 2025-01-01  135.1
253 2025-02-01  135.6
254 2025-03-01  136.1
255 2025-04-01  137.7
256 2025-05-01  138.0

[257 rows x 2 columns]


### Producing RHPI tables

Using CPIH to adjust HPI for inflation for each base 

In [ ]:

#now we will generate RHPI table with 2006,2020 base

RHPI_2015 = """ DROP TABLE IF EXISTS RHPI_2015;
CREATE TABLE RHPI_2015 AS
SELECT ab.Date, ab.District, ab.Region, (ab."Index"/c.B ) * 100 AS RHPI, (ab.DetachedIndex/c.B) * 100 AS RHPI_Detached,
(ab.SemiDetachedIndex/c.B) * 100 AS RHPI_Semi_Detached,(ab.TerracedIndex/c.B) * 100 AS RHPI_Terraced, (ab.FlatIndex/c.B) * 100 AS RHPI_Flat
From uk_hpi_2015 AS ab
LEFT JOIN CPIH_2015_J AS c
ON ab.Date = STRFTIME('%Y-%m-%d', c.A) 
WHERE ab.Date > '2014-12-30'; """

cursor.executescript(RHPI_2015)
conn.commit()

check_join_1 = """
SELECT *
FROM RHPI_2015
"""

RHPI_2020 = """ DROP TABLE IF EXISTS RHPI_2020;
CREATE TABLE RHPI_2020 AS
SELECT ab.Date, ab.District, ab.Region, (ab."Index"/c.B ) * 100 AS RHPI, (ab.DetachedIndex/c.B) * 100 AS RHPI_Detached,
(ab.SemiDetachedIndex/c.B) * 100 AS RHPI_Semi_Detached,(ab.TerracedIndex/c.B) * 100 AS RHPI_Terraced, (ab.FlatIndex/c.B) * 100 AS RHPI_Flat
From uk_hpi_2020 AS ab
LEFT JOIN CPIH_2020 AS c
ON ab.Date = STRFTIME('%Y-%m-%d', c.A) 
WHERE ab.Date > '2019-12-30'; """

cursor.executescript(RHPI_2020)
conn.commit()

check_join_2 = """
SELECT *
FROM RHPI_2020
"""

RHPI_2005 = """ DROP TABLE IF EXISTS RHPI_2005;
CREATE TABLE RHPI_2005 AS
SELECT ab.Date, ab.District, ab.Region, (ab."Index"/c.B ) * 100 AS RHPI, (ab.DetachedIndex/c.B) * 100 AS RHPI_Detached,
(ab.SemiDetachedIndex/c.B) * 100 AS RHPI_Semi_Detached,(ab.TerracedIndex/c.B) * 100 AS RHPI_Terraced, (ab.FlatIndex/c.B) * 100 AS RHPI_Flat
From uk_hpi_2005 AS ab
LEFT JOIN CPIH_2005 AS c
ON ab.Date = STRFTIME('%Y-%m-%d', c.A) 
WHERE ab.Date > '2004-12-30'; """

cursor.executescript(RHPI_2005)
conn.commit()

check_join_3 = """
SELECT *
FROM RHPI_2005
"""

# creating a unioned table of all 3 bases
# This will allow us to easily visualize the RHPI data 

union = """ SELECT *,'2005' AS BaseYear               
FROM RHPI_2005
UNION ALL
SELECT *,'2015' AS 'BaseYear'
FROM RHPI_2015
UNION ALL
SELECT *, '2020' AS 'BaseYear'
FROM RHPI_2020; """

print(pd.read_sql_query(union, conn))


df6 = pd.read_sql_query(union, conn)
df6.to_csv('/Users/bilallittle/Desktop/RHPI_union.csv', index=False)


              Date District                    Region        RHPI  \
0       2005-01-01     Adur                South East  100.000000   
1       2005-02-01     Adur                South East   95.444094   
2       2005-03-01     Adur                South East   96.508980   
3       2005-04-01     Adur                South East   96.752739   
4       2005-05-01     Adur                South East   98.412116   
...            ...      ...                       ...         ...   
126550  2024-11-01     York  Yorkshire and The Humber  100.254136   
126551  2024-12-01     York  Yorkshire and The Humber   98.404080   
126552  2025-01-01     York  Yorkshire and The Humber   97.516668   
126553  2025-02-01     York  Yorkshire and The Humber   96.076478   
126554  2025-03-01     York  Yorkshire and The Humber   97.191668   

        RHPI_Detached  RHPI_Semi_Detached  RHPI_Terraced   RHPI_Flat BaseYear  
0          100.000000          100.000000     100.000000  100.000000     2005  
1          

### BOE interest rates

We are adding adjusted BOE interest rates to our visual time series as an option for a baseline.

This will follow the same procces as RHPI (rebasing and adjusting)

# 1) Loading and rebasing

In [15]:

#For my next step I want to use Bank of england interest rates to produce an Index which can be used as a comparison for the RHPI

#downloading bank of england interest rate changes data
df_6 = pd.read_csv("Bank Rate history and data  Bank of England Database.csv")

try:
    df_6.to_sql(name='BOE_ir', con=conn, if_exists='replace', index=False)
    print("Data successfully loaded to SQLite!")
except Exception as e:
    print(f"Error: {e}")

print(pd.read_sql_query(""" SELECT * FROM BOE_ir """ ,conn))

#creating a table with the current interest rate for the first of each month from interest rate change data

cursor.execute("DROP INDEX IF EXISTS idx_boe_ir_date;")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_boe_ir_date ON BOE_ir(Date_Changed);")
conn.commit()

rates = """ DROP TABLE IF EXISTS BOE_Interest_Rates;
CREATE TABLE BOE_Interest_Rates AS
WITH Months AS (SELECT Date from RHPI_2005) 
SELECT DISTINCT Months.Date,(
        SELECT Rate
        FROM BOE_ir
        WHERE BOE_ir.Date_Changed <= Months.Date
        ORDER BY BOE_ir.Date_Changed DESC
        LIMIT 1
    ) AS current_rate
FROM
     Months
ORDER BY Months.Date; """

cursor.executescript(rates)
conn.commit()

rates_check = """SELECT *
FROM BOE_Interest_Rates"""

#print(pd.read_sql_query(rates_check, conn))

# Simple approach to create Savings_2005 table and rebased tables

# Step 1: Create Savings_2005 table from query results
savings_2005_query = """
WITH ordered_rates AS (
    SELECT 
        Date,
        current_rate,
        POWER(1 + current_rate/100.0, 1.0/12.0) as monthly_factor,
        ROW_NUMBER() OVER (ORDER BY Date) as row_num
    FROM BOE_Interest_Rates
    WHERE Date >= '2005-01-01'
    ORDER BY Date
),
running_product AS (
    SELECT 
        Date,
        monthly_factor,
        row_num,
        100.0 as base_value
    FROM ordered_rates
    WHERE row_num = 1
    
    UNION ALL
    
    SELECT 
        o.Date,
        o.monthly_factor,
        o.row_num,
        r.base_value * o.monthly_factor
    FROM ordered_rates o
    JOIN running_product r ON o.row_num = r.row_num + 1
)
SELECT Date, ROUND(base_value, 2) as Index_Value
FROM running_product
ORDER BY Date
"""

cursor.executescript(savings_2005_query)
conn.commit()
print(pd.read_sql_query(savings_2005_query, conn))

# Execute in Jupyter:
df_savings_2005 = pd.read_sql_query(savings_2005_query, conn)
df_savings_2005.to_sql('Savings_2005', conn, if_exists='replace', index=False)

# Step 2: Create rebased table for Jan 2015
rebase_2015_query = """DROP TABLE IF EXISTS Savings_2015_Rebased;
CREATE TABLE Savings_2015_Rebased AS
SELECT 
    Date,
    ROUND((Index_Value / (SELECT Index_Value FROM Savings_2005 WHERE Date = '2015-01-01')) * 100, 2) as Index_Value
FROM Savings_2005
WHERE Date >= '2015-01-01'
ORDER BY Date
"""
cursor.executescript(rebase_2015_query)
conn.commit()
print(pd.read_sql_query("""SELECT * FROM Savings_2015_Rebased""", conn))

# Step 3: Create rebased table for Jan 2020
rebase_2020_query = """DROP TABLE IF EXISTS Savings_2020_Rebased;
CREATE TABLE Savings_2020_Rebased AS
SELECT 
    Date,
    ROUND((Index_Value / (SELECT Index_Value FROM Savings_2005 WHERE Date = '2020-01-01')) * 100, 2) as Index_Value
FROM Savings_2005
WHERE Date >= '2020-01-01'
ORDER BY Date
"""

cursor.executescript(rebase_2020_query)
conn.commit()
#print(pd.read_sql_query("""SELECT * FROM Savings_2020_Rebased""", conn))


Data successfully loaded to SQLite!
    Date_Changed   Rate
0     2025-05-08   4.25
1     2025-02-06   4.50
2     2024-11-07   4.75
3     2024-08-01   5.00
4     2023-08-03   5.25
..           ...    ...
251   1975-03-10  10.25
252   1975-02-17  10.50
253   1975-02-10  10.75
254   1975-01-27  11.00
255   1975-01-20  11.25

[256 rows x 2 columns]
           Date  Index_Value
0    2005-01-01       100.00
1    2005-02-01       100.39
2    2005-03-01       100.78
3    2005-04-01       101.17
4    2005-05-01       101.56
..          ...          ...
238  2024-11-01       142.29
239  2024-12-01       142.85
240  2025-01-01       143.40
241  2025-02-01       143.95
242  2025-03-01       144.48

[243 rows x 2 columns]
           Date  Index_Value
0    2015-01-01       100.00
1    2015-02-01       100.04
2    2015-03-01       100.08
3    2015-04-01       100.12
4    2015-05-01       100.17
..          ...          ...
118  2024-11-01       114.09
119  2024-12-01       114.54
120  2025-01-01    

# Adjusting BOE using CPIH

In [16]:

# Queries to adjust savings indexes for CPIH inflation data
# Using matching CPIH base years: CPIH_2005 for Savings_2005, etc.

# 1. CPIH-adjusted Savings Index (2005 base)

Savings_2005 = """ DROP TABLE IF EXISTS Savings_2005_Real;
CREATE TABLE Savings_2005_Real AS
SELECT 
    s.Date,
    ROUND((s.Index_Value / c.B) * 100, 2) as Real_Index_Value,
    2005 AS BaseYear
FROM Savings_2005 s
JOIN CPIH_2005 c ON s.Date = c.A
ORDER BY s.Date;
"""
cursor.executescript(Savings_2005)
conn.commit()
#print(pd.read_sql_query("""SELECT * FROM Savings_2005_Real""", conn))

# 2. CPIH-adjusted Savings Index (2015 base)
Savings_2015_Real = """ DROP TABLE IF EXISTS Savings_2015_Real; 
CREATE TABLE Savings_2015_Real AS
SELECT 
    s.Date,
    ROUND((s.Index_Value / c.B) * 100, 2) as Real_Index_Value,
    2015 AS BaseYear
FROM Savings_2015_Rebased s
JOIN CPIH_2015_J c ON s.Date = c.A
ORDER BY s.Date;"""

cursor.executescript(Savings_2015_Real)
conn.commit()
#print(pd.read_sql_query("""SELECT * FROM Savings_2015_Real""", conn))

# 3. CPIH-adjusted Savings Index (2020 base)
Savings_2020_Real = """ DROP TABLE IF EXISTS Savings_2020_Real;
CREATE TABLE Savings_2020_Real AS
SELECT 
    s.Date,
    ROUND((s.Index_Value / c.B) * 100, 2) as Real_Index_Value,
    2020 AS BaseYear
FROM Savings_2020_Rebased s
JOIN CPIH_2020 c ON s.Date = c.A
ORDER BY s.Date; """

cursor.executescript(Savings_2020_Real)
conn.commit()
#print(pd.read_sql_query("""SELECT * FROM Savings_2020_Real""", conn))

#We now have 3 Savings tables with CPIH adjustments for each base year


# snp500

Following the same steps to produce adjusted snp500 as our final baseline

### 1) Loading and rebasing


In [17]:

#Now we will begin the process of producing snp500 indexes for each base year

#downloading snp500 data
df7 = pd.read_csv("snp500.csv")

try:
    df7.to_sql(name='snp500_o', con=conn, if_exists='replace', index=False)
    print("Data successfully loaded to SQLite!")
except Exception as e:
    print(f"Error: {e}")

#downloading dividend data
df8 = pd.read_csv("Dividends.csv")

try:
    df8.to_sql(name='dividends', con=conn, if_exists='replace', index=False)
    print("Data successfully loaded to SQLite!")
except Exception as e:
    print(f"Error: {e}")

#joining snp500 and dividends tables 
#the calculation in the select is combining snp%change and dividend rate to get overall percentage change per month

join_snp = """ DROP TABLE IF EXISTS snp500;
CREATE TABLE snp500 AS
SELECT o.Date, ((1 + o.snp_p_change) * (1 + (d.Dividend / (12 * o.Price * 1000))) - 1) AS ov_p_return
FROM snp500_o AS o
LEFT JOIN dividends AS d
ON o.Date = d.Date 
;"""

cursor.executescript(join_snp)
conn.commit()
#checking joined table
#print(pd.read_sql_query(""" SELECT * FROM snp500; """, conn))


#now we will calculate indexes for our 3 Base years
snp500_2005 = """WITH RECURSIVE index_calc_2005 AS (
    -- Base case: January 2005 = 100
    SELECT 
        Date,
        ov_p_return,
        100.0 as index_value
    FROM snp500
    WHERE Date = '2005-01-01'
    
    UNION ALL
    
    -- Recursive case: multiply previous index by (1 + monthly_return)
    SELECT 
        s.Date,
        s.ov_p_return,
        i.index_value * (1 + COALESCE(s.ov_p_return, 0)) as index_value
    FROM snp500 s
    INNER JOIN index_calc_2005 i ON s.Date = (
        SELECT MIN(s2.Date) 
        FROM snp500 s2 
        WHERE s2.Date > i.Date AND s2.Date >= '2005-01-01'
    )
    WHERE s.Date >= '2005-01-01'
)
SELECT 
    Date,
    ov_p_return,
    ROUND(index_value, 2) as index_2005_base
FROM index_calc_2005
ORDER BY Date ; """

cursor.executescript(snp500_2005)
conn.commit()

df_snp_2005 = pd.read_sql_query(snp500_2005, conn)
df_snp_2005.to_sql('snp500_2005', conn, if_exists='replace', index=False)

#checking snp500_2005 table
#print(pd.read_sql_query(""" SELECT * FROM snp500_2005; """, conn))


#INDEX WITH BASE JANUARY 2015 (Base = 100)
snp500_2015 = """
WITH RECURSIVE index_calc_2015 AS (
    -- Base case: January 2015 = 100
    SELECT 
        Date,
        ov_p_return,
        100.0 as index_value
    FROM snp500
    WHERE Date = '2015-01-01'
    
    UNION ALL
    
    -- Recursive case: multiply previous index by (1 + monthly_return)
    SELECT 
        s.Date,
        s.ov_p_return,
        i.index_value * (1 + COALESCE(s.ov_p_return, 0)) as index_value
    FROM snp500 s
    INNER JOIN index_calc_2015 i ON s.Date = (
        SELECT MIN(s2.Date) 
        FROM snp500 s2 
        WHERE s2.Date > i.Date AND s2.Date >= '2015-01-01'
    )
    WHERE s.Date >= '2015-01-01'
)
SELECT 
    Date,
    ov_p_return,
    ROUND(index_value, 2) as index_2015_base
FROM index_calc_2015
ORDER BY Date; """

cursor.executescript(snp500_2015)
conn.commit()

df_snp_2015 = pd.read_sql_query(snp500_2015, conn)
df_snp_2015.to_sql('snp500_2015', conn, if_exists='replace', index=False)

#checking snp500_2015 table
#print(pd.read_sql_query(""" SELECT * FROM snp500_2015; """, conn))

#INDEX WITH BASE JANUARY 2020 (Base = 100)
snp500_2020 = """
WITH RECURSIVE index_calc_2020 AS (
    -- Base case: January 2020 = 100
    SELECT 
        Date,
        ov_p_return,
        100.0 as index_value
    FROM snp500
    WHERE Date = '2020-01-01'
    
    UNION ALL
    
    -- Recursive case: multiply previous index by (1 + monthly_return)
    SELECT 
        s.Date,
        s.ov_p_return,
        i.index_value * (1 + COALESCE(s.ov_p_return, 0)) as index_value
    FROM snp500 s
    INNER JOIN index_calc_2020 i ON s.Date = (
        SELECT MIN(s2.Date) 
        FROM snp500 s2 
        WHERE s2.Date > i.Date AND s2.Date >= '2020-01-01'
    )
    WHERE s.Date >= '2020-01-01'
)
SELECT 
    Date,
    ov_p_return,
    ROUND(index_value, 2) as index_2020_base
FROM index_calc_2020
ORDER BY Date; """

cursor.executescript(snp500_2020)
conn.commit()
df_snp_2020 = pd.read_sql_query(snp500_2020, conn)
df_snp_2020.to_sql('snp500_2020', conn, if_exists='replace', index=False)

#checking snp500_2020 table
print(pd.read_sql_query(""" SELECT * FROM snp500_2020; """, conn))

Data successfully loaded to SQLite!
Data successfully loaded to SQLite!
          Date  ov_p_return  index_2020_base
0   2020-01-01    -0.078512           100.00
1   2020-02-01    -0.127872            87.21
2   2020-03-01     0.132784            98.79
3   2020-04-01     0.052607           103.99
4   2020-05-01     0.021689           106.25
..         ...          ...              ...
61  2025-02-01    -0.058821           208.51
62  2025-03-01    -0.008751           206.69
63  2025-04-01     0.061345           219.37
64  2025-05-01     0.051340           230.63
65  2025-06-01          NaN           230.63

[66 rows x 3 columns]


### Adjusting snp500 with CPIH

In [18]:

# Adjust S&P 500 2005 Index using CPIH_2005
query_adjust_2005 = """
DROP TABLE IF EXISTS snp500_real_2005_base;
CREATE TABLE snp500_real_2005_base AS
SELECT 
    s.Date,
    ROUND((s.index_2005_base / c.B) * 100, 2) as real_index_2005_base
FROM snp500_2005 s
LEFT JOIN CPIH_2005 c ON s.Date = c.A
WHERE c.B IS NOT NULL
ORDER BY s.Date;
"""

cursor.executescript(query_adjust_2005)
conn.commit()

#Adjusting all snp tables for CPIH

# Adjust S&P 500 2015 Index using CPIH_2015_J
query_adjust_2015 = """
CREATE TABLE IF NOT EXISTS snp500_real_2015_base AS
SELECT 
    s.Date,
    ROUND((s.index_2015_base / c.B) * 100, 2) as real_index_2015_base
FROM snp500_2015 s
LEFT JOIN CPIH_2015_J c ON s.Date = c.A
WHERE c.B IS NOT NULL
ORDER BY s.Date;
"""

cursor.executescript(query_adjust_2015)
conn.commit()

# Adjust S&P 500 2020 Index using CPIH_2020
query_adjust_2020 = """
CREATE TABLE IF NOT EXISTS snp500_real_2020_base AS
SELECT 
    s.Date,
    ROUND((s.index_2020_base / c.B) * 100, 2) as real_index_2020_base
FROM snp500_2020 s
LEFT JOIN CPIH_2020 c ON s.Date = c.A
WHERE c.B IS NOT NULL
ORDER BY s.Date;
"""

cursor.executescript(query_adjust_2020)
conn.commit()

print(pd.read_sql_query(""" SELECT * FROM snp500_real_2005_base; """, conn))
print(pd.read_sql_query(""" SELECT * FROM snp500_real_2015_base; """, conn))
print(pd.read_sql_query(""" SELECT * FROM snp500_real_2020_base; """, conn))

           Date  real_index_2005_base
0    2005-01-01                100.00
1    2005-02-01                 97.91
2    2005-03-01                 95.75
3    2005-04-01                 98.41
4    2005-05-01                 98.21
..          ...                   ...
240  2025-01-01                449.96
241  2025-02-01                421.92
242  2025-03-01                416.70
243  2025-04-01                437.12
244  2025-05-01                458.56

[245 rows x 2 columns]
           Date  real_index_2015_base
0    2015-01-01                100.00
1    2015-02-01                 97.86
2    2015-03-01                 98.91
3    2015-04-01                 99.77
4    2015-05-01                 97.74
..          ...                   ...
120  2025-01-01                258.93
121  2025-02-01                242.80
122  2025-03-01                239.79
123  2025-04-01                251.54
124  2025-05-01                263.89

[125 rows x 2 columns]
          Date  real_index_2020_base
0  

# Unioned table

Creating final unioned table for data vizualization in Tableau

In [28]:
conn = sqlite3.connect("house_prices_1.db")
cursor = conn.cursor()

# Storing RHPI union as sql table
df_00 = pd.read_sql_query(union, conn)
df_00.to_sql('union_1', conn, if_exists='replace', index=False)

# finally creating a union of RHPI,Savings,snp data for vizualization

# All tables have different columns/ different numbers of columns 
# so this makes creating a combined table slightly harder

# Will start by viewing all columns in each table

tables = ['union_1', 'Savings_2005_Real', 'Savings_2015_Real', 
         'Savings_2020_Real', 'snp500_real_2005_base', 
         'snp500_real_2015_base', 'snp500_real_2020_base']

for table in tables:
    print(f"\n{table}:")
    result = pd.read_sql(f"PRAGMA table_info({table})", conn)
    print(result[['name', 'type']]) 

union_all = """-- Drop existing table if it exists
DROP TABLE IF EXISTS all_combined;

-- Create the combined table with all columns
CREATE TABLE all_combined (
    Date TEXT,
    District TEXT,
    Region TEXT,
    RHPI REAL,
    RHPI_Detached REAL,
    RHPI_Semi_Detached REAL,
    RHPI_Terraced REAL,
    RHPI_Flat REAL,
    Real_Index_Value REAL,
    real_index_2005_base REAL,
    real_index_2015_base REAL,
    real_index_2020_base REAL,
    BaseYear TEXT,
    source_table TEXT
);

-- Insert from each table
INSERT INTO all_combined (Date, District, Region, RHPI, RHPI_Detached, RHPI_Semi_Detached, RHPI_Terraced, RHPI_Flat, BaseYear, source_table)
SELECT Date, District, Region, RHPI, RHPI_Detached, RHPI_Semi_Detached, RHPI_Terraced, RHPI_Flat, BaseYear, 'union_1' FROM union_1;

INSERT INTO all_combined (Date, Real_Index_Value, BaseYear, source_table)
SELECT Date, Real_Index_Value, BaseYear, 'Savings_2005_Real' FROM Savings_2005_Real;

INSERT INTO all_combined (Date, Real_Index_Value, BaseYear, source_table)
SELECT Date, Real_Index_Value, BaseYear, 'Savings_2015_Real' FROM Savings_2015_Real;

INSERT INTO all_combined (Date, Real_Index_Value, BaseYear, source_table)
SELECT Date, Real_Index_Value, BaseYear, 'Savings_2020_Real' FROM Savings_2020_Real;

INSERT INTO all_combined (Date, real_index_2005_base, source_table)
SELECT Date, real_index_2005_base, 'snp500_real_2005_base' FROM snp500_real_2005_base;

INSERT INTO all_combined (Date, real_index_2015_base, source_table)
SELECT Date, real_index_2015_base, 'snp500_real_2015_base' FROM snp500_real_2015_base;

INSERT INTO all_combined (Date, real_index_2020_base, source_table)
SELECT Date, real_index_2020_base, 'snp500_real_2020_base' FROM snp500_real_2020_base;
"""
cursor.executescript(union_all)
conn.commit()

# Checking the combined table
union_all_1 = """ SELECT * FROM all_combined; """
df_combined = pd.read_sql_query(union_all_1, conn)
df_combined.to_csv('all_combined.csv', index=False)


conn.close()
print("\nDatabase connection closed.")


union_1:
                 name  type
0                Date  TEXT
1            District  TEXT
2              Region  TEXT
3                RHPI  REAL
4       RHPI_Detached  REAL
5  RHPI_Semi_Detached  REAL
6       RHPI_Terraced  REAL
7           RHPI_Flat  REAL
8            BaseYear  TEXT

Savings_2005_Real:
               name  type
0              Date  TEXT
1  Real_Index_Value      
2          BaseYear      

Savings_2015_Real:
               name  type
0              Date  TEXT
1  Real_Index_Value      
2          BaseYear      

Savings_2020_Real:
               name  type
0              Date  TEXT
1  Real_Index_Value      
2          BaseYear      

snp500_real_2005_base:
                   name  type
0                  Date  TEXT
1  real_index_2005_base      

snp500_real_2015_base:
                   name  type
0                  Date  TEXT
1  real_index_2015_base      

snp500_real_2020_base:
                   name  type
0                  Date  TEXT
1  real_index_2020_base   